In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import requests
from datetime import datetime
import pytz

# Stack-Society 2025

# Endereço IP da máquina virtual onde está rodando o serviço STH (Short-Term Historic)
ENDERECO_IP = "20.64.236.3"  # Altere para o IP da sua VM
PORTA_STH = 8666  # Porta do serviço STH
HOST_DASH = "127.0.0.1"  # Host onde o Dash vai rodar (localhost)

# Função para buscar os dados de enchente do serviço STH
def buscar_enchente(qtd_registros):
    # Monta a URL para pegar os últimos N registros do atributo 'enchente' da entidade Lamp
    url = f"http://{ENDERECO_IP}:{PORTA_STH}/STH/v1/contextEntities/type/Lamp/id/urn:ngsi-ld:stack-enchente:001/attributes/enchente?lastN={qtd_registros}"
    cabecalhos = {
        'fiware-service': 'smart',    # Header obrigatório para FIWARE
        'fiware-servicepath': '/'     # Caminho do serviço FIWARE
    }
    resposta = requests.get(url, headers=cabecalhos)  # Faz a requisição GET
    if resposta.status_code == 200:
        dados = resposta.json()  # Converte a resposta para JSON
        try:
            # Extrai a lista de valores retornada do JSON
            valores = dados['contextResponses'][0]['contextElement']['attributes'][0]['values']
            return valores
        except KeyError as erro:
            print(f"Erro de chave: {erro}")  # Caso a estrutura do JSON mude ou não tenha a chave
            return []
    else:
        print(f"Erro ao acessar {url}: {resposta.status_code}")  # Caso a requisição falhe
        return []

# Função para converter timestamps em UTC para horário de São Paulo
def converter_para_sao_paulo(lista_timestamps):
    fuso_utc = pytz.utc
    fuso_sp = pytz.timezone('America/Sao_Paulo')
    timestamps_convertidos = []
    for tempo in lista_timestamps:
        try:
            # Remove o 'T' e 'Z' do timestamp para facilitar o parsing
            tempo = tempo.replace('T', ' ').replace('Z', '')
            # Tenta converter com milissegundos
            convertido = fuso_utc.localize(datetime.strptime(tempo, '%Y-%m-%d %H:%M:%S.%f')).astimezone(fuso_sp)
        except ValueError:
            # Caso não tenha milissegundos, tenta outra formatação
            convertido = fuso_utc.localize(datetime.strptime(tempo, '%Y-%m-%d %H:%M:%S')).astimezone(fuso_sp)
        timestamps_convertidos.append(convertido)
    return timestamps_convertidos

# Recebe do usuário quantos dados ele quer requisitar
lastn = int(input('Quantos dados você deseja requisitar:'))

# Inicializa o app Dash
app = dash.Dash(__name__)

# Layout da aplicação, com título, gráfico, armazenamento local e temporizador de atualização
app.layout = html.Div([
    html.H1('Visualizador de Enchente'),  # Título da página
    dcc.Graph(id='grafico-enchente'),     # Gráfico que vai mostrar o nível de enchente
    dcc.Store(id='store-enchente', data={'timestamps': [], 'valores_enchente': []}),  # Armazena dados localmente
    dcc.Interval(
        id='intervalo-atualizacao',
        interval=10*1000,  # Intervalo de atualização a cada 10 segundos (em milissegundos)
        n_intervals=0
    )
])

# Callback que atualiza os dados do nível de enchente a cada intervalo de tempo
@app.callback(
    Output('store-enchente', 'data'),   # Atualiza o armazenamento local com novos dados
    Input('intervalo-atualizacao', 'n_intervals'),  # Disparado a cada intervalo definido
    State('store-enchente', 'data')     # Pega os dados atuais armazenados
)
def atualizar_dados(n_intervals, dados_enchente):
    dados = buscar_enchente(lastn)  # Busca os últimos dados do serviço
    if dados:
        # Extrai os valores numéricos do nível da enchente
        valores = [float(item['attrValue']) for item in dados]
        # Extrai os timestamps recebidos
        tempos = [item['recvTime'] for item in dados]
        # Converte os timestamps para horário de São Paulo
        tempos = converter_para_sao_paulo(tempos)
        # Adiciona os novos dados na lista existente (acumula)
        dados_enchente['timestamps'].extend(tempos)
        dados_enchente['valores_enchente'].extend(valores)
    return dados_enchente  # Retorna os dados atualizados para o armazenamento

# Callback que atualiza o gráfico sempre que os dados no armazenamento mudam
@app.callback(
    Output('grafico-enchente', 'figure'),  # Atualiza o gráfico
    Input('store-enchente', 'data')        # Quando os dados são atualizados
)
def atualizar_grafico(dados_enchente):
    if dados_enchente['timestamps'] and dados_enchente['valores_enchente']:
        # Cria a linha do gráfico com os dados coletados
        curva_enchente = go.Scatter(
            x=dados_enchente['timestamps'],        # Eixo X: horários convertidos
            y=dados_enchente['valores_enchente'],  # Eixo Y: nível da enchente
            mode='lines+markers',                   # Mostra linha e pontos
            name='Enchente',
            line=dict(color='royalblue')            # Cor da linha
        )
        # Configura o layout do gráfico com títulos e hover
        figura = go.Figure(data=[curva_enchente])
        figura.update_layout(
            title='Nível de Enchente ao Longo do Tempo',
            xaxis_title='Horário',
            yaxis_title='Nível de Enchente',
            hovermode='closest'  # Mostra o ponto mais próximo ao passar o mouse
        )
    else:
        # Caso não tenha dados, retorna figura vazia
        figura = go.Figure()
    return figura

# Ponto de entrada para rodar o app Dash
if __name__ == '__main__':
    app.run(debug=True, host=HOST_DASH, port=8050)



[2025-06-03 14:38:50,682] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\vitor\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "c:\Users\vitor\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\vitor\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\vitor\AppData\Local\Prog